In [1]:
import numpy as np
hex_to_bin = {'0': '0000', \
              '1': '0001', \
              '2': '0010', \
              '3': '0011', \
              '4': '0100', \
              '5': '0101', \
              '6': '0110', \
              '7': '0111', \
              '8': '1000', \
              '9': '1001', \
              'A': '1010', \
              'B': '1011', \
              'C': '1100', \
              'D': '1101', \
              'E': '1110', \
              'F': '1111'}

def calculate(array, operator):
    if operator == 0:
        return np.sum(array)
    elif operator == 1:
        return np.product(array)
    elif operator == 2:
        return np.min(array)
    elif operator == 3:
        return np.max(array)
    elif operator == 5:
        return int(array[0] > array[1])
    elif operator == 6:
        return int(array[0] < array[1])
    elif operator == 7:
        return 1 if array[0] == array[1] else 0

def binary_to_decimal(binary_number):
    #Zahl umdrehen
    binary_number = binary_number[::-1]
    exponent = 0
    decimal = 0
    for cifer in binary_number:
        decimal += int(cifer)*2**exponent
        exponent += 1
        
    return decimal 

class DAY_16:
    def __init__(self, hex_string, verbose=True, test=-1):
        self.__hex_string = hex_string
        self.__verbose = verbose
        self.__number = []
        self.__packet_version = 0
        self.__test = test

    def convert_to_binary_string(self):
        self.__binary_string = ""
        for element in self.__hex_string:
            self.__binary_string += hex_to_bin[element]
            
        if self.__verbose: print(len(self.__binary_string), self.__binary_string)
        
    def __get_packet_version(self, binary_string):
        packet_version = binary_to_decimal(binary_string[:3])
        
        if self.__verbose: print("Packet Version", packet_version)
        return packet_version, binary_string[3:]
        
    def __get_packet_type_id(self, binary_string):
        packet_type_id = binary_to_decimal(binary_string[:3])
        
        if self.__verbose: print("Packet Type ID", packet_type_id)
        return packet_type_id, binary_string[3:]
        
    def __get_number(self, current_binary_string=""):
        binary_text = ""
        while True:
            binary_text += current_binary_string[1:5]
            if current_binary_string[0] == '0': 
                #Hier müssen die letzten ungelabelten Bits entfernt werden. Dazu muss die Anzahl gesamten Bits übergeben werden
                break
            else:
                current_binary_string = current_binary_string[5:]

        return [binary_to_decimal(binary_text), current_binary_string[5:]]
        
    def __parse_string(self, binary_string):
        packet_version, binary_string = self.__get_packet_version(binary_string)
        self.__packet_version += packet_version
        packet_type_id, binary_string = self.__get_packet_type_id(binary_string)
        #print("Packet_Type_ID", packet_type_id)
        #print("binary_string", binary_string)
        #Wenn es sich bei dem aktuellen String um eine Zahl handelt, bestimme die Zahl. Nach einer Zahl ist das Parsen
        #an dieser Stelle Vorbei
        if packet_type_id == 4:
            number, binary_string = self.__get_number(binary_string)
            return binary_string, number

        #Falls es ein anderer Operator ist, bestimme dessen Typen
        elif len(binary_string) != 0 and binary_string[0] == '0':#Die nächsten 15 Bits
            length_type_id = binary_string[0]
            binary_string = binary_string[1:]
            next_bits = binary_to_decimal(binary_string[:15])
            binary_string = binary_string[15:]
            if self.__verbose: print(next_bits)
            #Gehe die nächsten n-Bits durch
            #binary_string_next_bits = binary_string[:next_bits]
            binary_string_to_remove = binary_string[:next_bits]
            numbers = []
            while len(binary_string_to_remove) > 6:
                binary_string_to_remove, numbers_append = self.__parse_string(binary_string_to_remove)
                numbers.append(int(numbers_append))
            self.__number.append(calculate(numbers, packet_type_id))
            return binary_string[next_bits:], calculate(numbers, packet_type_id)

        elif len(binary_string) != 0 and binary_string[0] == '1':#Die nächsten 11 Bits
            binary_string = binary_string[1:]
            number_of_sub_packages = binary_to_decimal(binary_string[:11])
            if self.__verbose: print("Number of subpackages", number_of_sub_packages)
            binary_string = binary_string[11:]
            numbers = []
            for iter in range(number_of_sub_packages):
                binary_string, numbers_append = self.__parse_string(binary_string)
                numbers.append(int(numbers_append))
            self.__number.append(calculate(numbers, packet_type_id))
            return binary_string, calculate(numbers, packet_type_id)
        else:
            return binary_string, 0

    def read_hex_string(self):
        print("All operations applied to number literals: ", self.__parse_string(self.__binary_string)[1])
        print("Sum Packet Versions", self.__packet_version)
        if self.__test > 0:
            print("Test erfolgreich?", self.__packet_version == self.__test)
        
#day16_test_1a = DAY_16("D2FE28", False)
#day16_test_1a.convert_to_binary_string()
#day16_test_1a.read_hex_string()

#day16_test_1b = DAY_16("38006F45291200", False)
#day16_test_1b.convert_to_binary_string()
#day16_test_1b.read_hex_string()

#day16_test_1c = DAY_16("EE00D40C823060", False)
#day16_test_1c.convert_to_binary_string()
#day16_test_1c.read_hex_string()

#day16_test_1d = DAY_16("8A004A801A8002F478", False, test=16)
#day16_test_1d.convert_to_binary_string()
#day16_test_1d.read_hex_string()

#day16_test_1e = DAY_16("620080001611562C8802118E34", False, test=12)
#day16_test_1e.convert_to_binary_string()
#day16_test_1e.read_hex_string()

#day16_test_1f = DAY_16("C0015000016115A2E0802F182340", False, test=23)
#day16_test_1f.convert_to_binary_string()
#day16_test_1f.read_hex_string()

#day16_test_1g = DAY_16("A0016C880162017C3686B18A3D4780", False, test=31)
#day16_test_1g.convert_to_binary_string()
#day16_test_1g.read_hex_string()

#day16_test_2a = DAY_16("C200B40A82", False)
#day16_test_2a.convert_to_binary_string()
#day16_test_2a.read_hex_string()

#day16_test_2b = DAY_16("04005AC33890", False)
#day16_test_2b.convert_to_binary_string()
#day16_test_2b.read_hex_string()

#day16_test_2c = DAY_16("880086C3E88112", False)
#day16_test_2c.convert_to_binary_string()
#day16_test_2c.read_hex_string()

#day16_test_2d = DAY_16("CE00C43D881120", False)
#day16_test_2d.convert_to_binary_string()
#day16_test_2d.read_hex_string()

#D8005AC2A8F0 produces 1, because 5 is less than 15.
#day16_test_2 = DAY_16("D8005AC2A8F0", False)
#day16_test_2.convert_to_binary_string()
#day16_test_2.read_hex_string()

#F600BC2D8F produces 0, because 5 is not greater than 15.
#day16_test_2 = DAY_16("F600BC2D8F", False)
#day16_test_2.convert_to_binary_string()
#day16_test_2.read_hex_string()

#9C005AC2F8F0 produces 0, because 5 is not equal to 15.
#day16_test_2 = DAY_16("9C005AC2F8F0", False)
#day16_test_2.convert_to_binary_string()
#day16_test_2.read_hex_string()

#9C0141080250320F1802104A08 produces 1, because 1 + 3 = 2 * 2.
#day16_test_2 = DAY_16("9C0141080250320F1802104A08", False)
#day16_test_2.convert_to_binary_string()
#day16_test_2.read_hex_string()

hex_string_input = "0052E4A00905271049796FB8872A0D25B9FB746893847236200B4F0BCE5194401C9B9E3F9C63992C8931A65A1CCC0D222100511A00BCBA647D98BE29A397005E55064A9DFEEC86600BD002AF2343A91A1CCE773C26600D126B69D15A6793BFCE2775D9E4A9002AB86339B5F9AB411A15CCAF10055B3EFFC00BCCE730112FA6620076268CE5CDA1FCEB69005A3800D24F4DB66E53F074F811802729733E0040E5C5E5C5C8015F9613937B83F23B278724068018014A00588014005519801EC04B220116CC0402000EAEC03519801A402B30801A802138801400170A0046A800C10001AB37FD8EB805D1C266963E95A4D1A5FF9719FEF7FDB4FB2DB29008CD2BAFA3D005CD31EB4EF2EBE4F4235DF78C66009E80293AE9310D3FCBFBCA440144580273BAEE17E55B66508803C2E0087E630F72BCD5E71B32CCFBBE2800017A2C2803D272BCBCD12BD599BC874B939004B5400964AE84A6C1E7538004CD300623AC6C882600E4328F710CC01C82D1B228980292ECD600B48E0526E506F700760CCC468012E68402324F9668028200C41E8A30E00010D8B11E62F98029801AB88039116344340004323EC48873233E72A36402504CB75006EA00084C7B895198001098D91AE2190065933AA6EB41AD0042626A93135681A400804CB54C0318032200E47B8F71C0001098810D61D8002111B228468000E5269324AD1ECF7C519B86309F35A46200A1660A280150968A4CB45365A03F3DDBAE980233407E00A80021719A1B4181006E1547D87C6008E0043337EC434C32BDE487A4AE08800D34BC3DEA974F35C20100BE723F1197F59E662FDB45824AA1D2DDCDFA2D29EBB69005072E5F2EDF3C0B244F30E0600AE00203229D229B342CC007EC95F5D6E200202615D000FB92CE7A7A402354EE0DAC0141007E20C5E87A200F4318EB0C"
day16_puzzle = DAY_16(hex_string_input, False)
day16_puzzle.convert_to_binary_string()#
day16_puzzle.read_hex_string()

All operations applied to number literals:  18234816469452
Sum Packet Versions 986
